In [8]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pickle

from flask import Flask, render_template, request
app = Flask(__name__)

#Загрузка обученной модели нейронной сети
model_loaded = keras.saving.load_model('model\\NN3')

# Функция денормализации по yeojohnson
#  inverse of the Yeo-Johnson transformation
def yeojohnson_inverse(X_trans, lambda_):
  '''
  if X >= 0 and lambda_ == 0:
    X = np.exp(X_trans) - 1
  elif X >= 0 and lambda_ != 0:
      X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1
  elif X < 0 and lambda_ != 2:
      X = 1 - (-(2 - lambda_) * X_trans + 1) ** (1 / (2 - lambda_))
  elif X < 0 and lambda_ == 2:
      X = 1 - np.exp(-X_trans)
  '''

  if lambda_ == 0:
    X = np.exp(X_trans) - 1
  elif lambda_ != 0:
    X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1  

  return X


def print_params_for_NN(x1, x2, x3):
    if (x1 == "" or x2 == "" or x3 == ""):
        message = "ОШИБКА! Вы не ввели параметры."
    elif(float(x1) >= 0 and float(x2) >=0 and float(x3) >=0):
        message = f"x1= {x1}; x2= {x2} ; x3= {x3}"
    else:
        message = "ОШИБКА! Введенные значения должны быть больше или равны 0."
    return message

#Функция прогнозоа Y с помощью модели нейронной сети
def calculate_NN(x1, x2, x3):
    X_use = pd.DataFrame([[x1,x2,x3]],columns=['Работники, чел.','Наличие тракторов, шт.','Общая площадь земли, га'])
    X_use = np.array([[x1, x2, x3]])
    
    dummy_column = np.zeros(shape = (X_use.shape[0], 1))
    cols_indexes = [0, 3, 5] # Индексы фиктивных столбцов (для StandardScaler)
    X_use_6cols = np.insert(X_use, 0, dummy_column.T, axis = 1)
    X_use_6cols = np.insert(X_use_6cols, 3, dummy_column.T, axis = 1)
    X_use_6cols = np.insert(X_use_6cols, 5, dummy_column.T, axis = 1)
    
    # Нормализация по yeojohnson
    lamda_list = [0.013865117288407803,
                  -1.3939833064592597,
                  -0.0542790101104967,
                  0,
                  -0.030491839284671637,
                  0] # Список значений lambda для каждого столбца (у фиктивных столбцлв lambda= 0)
    
    X_use_6cols_normalized = X_use_6cols.copy()
    
    #Нормализация факторов по Yeo-Johnson
    X_use_6cols_normalized[:, 1] = stats.yeojohnson(X_use_6cols_normalized[:, 1], lmbda = lamda_list[1]) 
    X_use_6cols_normalized[:, 2] = stats.yeojohnson(X_use_6cols_normalized[:, 2], lmbda = lamda_list[2]) 
    X_use_6cols_normalized[:, 4] = stats.yeojohnson(X_use_6cols_normalized[:, 4], lmbda = lamda_list[4]) 
    
    # Загрузка standardscaler
    with open('scaler\\scaler.pkl','rb') as f:
        standardscaler = pickle.load(f)
    
    # Стандартизация факторов по StandardScaler
    X_use_6cols_standard = standardscaler.transform(X_use_6cols_normalized)
    
    # Удаление фиктивных столбцов
    X_use_standard = np.delete(X_use_6cols_standard, [5,3,0], axis = 1)
    
    #Прогноз Y по нормализованным и стандартизированным факторам
    y_pred_use_standard = model_loaded.predict(X_use_standard)
    print('y_pred_use_standard =', y_pred_use_standard)
    
    #Создание матрицы с   Y_pred  и пятью фиктивными столбцами 
    y_pred_use_standard_6cols = np.zeros(shape = (X_use.shape[0], 6))
    y_pred_use_standard_6cols[:, 0] = y_pred_use_standard[:, 0]
    
    #Дестандартизация Y_pred и сохранение только первого столбца из матрицы
    y_pred_use_standard_inv = standardscaler.inverse_transform(y_pred_use_standard_6cols)[:, 0]
    print('y_pred_use_standard_inv =', y_pred_use_standard_inv)
    
    #Денормализация Y_pred 
    y_pred_use_normalized_inv = y_pred_use_standard_inv.copy()
    y_pred_use_normalized_inv = yeojohnson_inverse(X_trans = y_pred_use_normalized_inv, lambda_ = lamda_list[0])
    print('y_pred_use_normalized_inv =', y_pred_use_normalized_inv)
    

    message = ": ".join(["Доход, тыс. руб.", str(np.round(y_pred_use_normalized_inv[0],2))]) 
    
    return message

@app.route("/", methods=["post", "get"])
def index():
    message = ''
    message2 = ''
    x1=0
    x2=0
    x3=0
    if request.method == "POST":
        x1 = request.form.get("x1")
        x2 = request.form.get("x2")
        x3 = request.form.get("x3")
        
        message = print_params_for_NN(x1, x2, x3) 
        if "ОШИБКА" not in message:
            message2 = calculate_NN(float(x1), float(x2), float(x3))
        
    return render_template("index.html", message=message, message2=message2, x1=x1, x2=x2, x3=x3)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2023 13:51:07] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 99ms/step


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [29/Apr/2023 13:51:15] "POST / HTTP/1.1" 200 -


y_pred_use_standard = [[1.1291225]]
y_pred_use_standard_inv = [10.49224892]
y_pred_use_normalized_inv = [17957.91823498]
